# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later
import sys
import os
import json
import pandas as pd
sys.path.insert(1, '/locale/data/jupyter/prando/notebook/sparqlthesis/')
import modules.evaluation as evaluation
from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-history7-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Literary Movements and physicists explorative search") 

Consider the following exploratory information need:

> investigate the literary movements and the poets. In particular, compare some physicists in terms of discovers, Nobel Prizes and education.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q43514`| theory of relativity | node |





Also consider

```
wd:Q43514 ?p ?obj .
```

is the BGP to retrieve all **properties of the theory of relativity**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:

1. Who invented the theory of relativity? (return the IRI and the label of the inventor).

2. Identify the BGP for physicist

3. Identify the BGP for Nobel Prize

4. Consider physicists born in Europe. For each european country return the number of physicists who won the Nobel Prize in Physics (the result set must be a list of triples country IRI, label and #physicists).

5. Return the countries which won at least once all the type of Nobel Prizes with the people born there. (the result set must be a list of couples country IRI and label). 

6. Amongst the physicists who won the Nobel Prize in Physics, find who had discovered or invented at least 3 things (the result set must be a list of triples physicist IRI, label and #discoveries ).

7. Amongst the physicists who won the Nobel Prize in Physics, find who taught in the university of his/her city of birth (the result set must be a list of elements of the form physicist IRI, label, university IRI and label).

In [3]:
## startup the evaluation
# setup the file and create the empty json
ipname = "h7.ipynb"
pt = os.getcwd()+os.sep+ipname
evaluation.setup(pt)

The index of this workflow is: 1_0


## Task 1

In [4]:
# look at the properties of the theory of relativity
queryString = """
SELECT  DISTINCT ?prop ?name 
WHERE {
    wd:Q43514 ?prop ?obj .
    ?prop sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('prop', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('prop', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('prop', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('prop', 'http://www

In [7]:
# use invetor
queryString = """
SELECT  DISTINCT ?obj ?name 
WHERE {
    wd:Q43514 wdt:P61 ?obj .
    ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q937'), ('name', 'Albert Einstein')]
1


In [8]:
### insert the result of TASK 1 in the file
og_uri = "http://www.wikidata.org/entity/Q937"
og_name = "Albert Einstein"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"1", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 1_0
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_0.json
JSON object updated


## Task 2

In [10]:
# list Einstein's occupations
queryString = """
SELECT  DISTINCT ?obj ?name 
WHERE {
    wd:Q937 wdt:P106 ?obj .
    ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q121594'), ('name', 'professor')]
[('obj', 'http://www.wikidata.org/entity/Q1231865'), ('name', 'pedagogue')]
[('obj', 'http://www.wikidata.org/entity/Q15980158'), ('name', 'non-fiction writer')]
[('obj', 'http://www.wikidata.org/entity/Q1622272'), ('name', 'university teacher')]
[('obj', 'http://www.wikidata.org/entity/Q169470'), ('name', 'physicist')]
[('obj', 'http://www.wikidata.org/entity/Q170790'), ('name', 'mathematician')]
[('obj', 'http://www.wikidata.org/entity/Q205375'), ('name', 'inventor')]
[('obj', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer')]
[('obj', 'http://www.wikidata.org/entity/Q37226'), ('name', 'teacher')]
[('obj', 'http://www.wikidata.org/entity/Q4964182'), ('name', 'philosopher')]
[('obj', 'http://www.wikidata.org/entity/Q901'), ('name', 'scientist')]
[('obj', 'http://www.wikidata.org/entity/Q19350898'), ('name', 'theoretical physicist')]
[('obj', 'http://www.wikidata.org/entity/Q16389557'), ('name

In [11]:
### insert the result of TASK 2 in the file
og_uri = "http://www.wikidata.org/entity/Q169470"
og_name = "physicist"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"2", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 1_0
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_0.json
JSON object updated


`Q169470` -> physicist

## Task 3

In [12]:
# look for object properties of Einstein
queryString = """
SELECT  DISTINCT ?pr ?name 
WHERE {
    wd:Q937 ?pr ?obj .
    ?pr sc:name ?name.
    filter(!isLiteral(?obj)).
}
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('pr', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('pr', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('pr', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('pr', 'http://www.wikidata.org/prop/direct/P1066'), ('name', 'student of')]
[('pr', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('pr', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('pr', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('pr', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('pr', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('pr', 'http://www.wi

In [13]:
# list the awards received
queryString = """
SELECT  DISTINCT ?obj ?name 
WHERE {
    wd:Q937 wdt:P166 ?obj .
    ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1419938'), ('name', 'Matteucci Medal')]
[('obj', 'http://www.wikidata.org/entity/Q14906020'), ('name', 'Foreign Member of the Royal Society')]
[('obj', 'http://www.wikidata.org/entity/Q15056034'), ('name', 'Pour le Mérite for Sciences and Arts')]
[('obj', 'http://www.wikidata.org/entity/Q25287226'), ('name', 'Honorary doctor of the University of Geneva')]
[('obj', 'http://www.wikidata.org/entity/Q28003'), ('name', 'Copley Medal')]
[('obj', 'http://www.wikidata.org/entity/Q28861731'), ('name', 'honorary doctor of the Hebrew University of Jerusalem')]
[('obj', 'http://www.wikidata.org/entity/Q38104'), ('name', 'Nobel Prize in Physics')]
[('obj', 'http://www.wikidata.org/entity/Q42309226'), ('name', 'honorary doctor of the Princeton University')]
[('obj', 'http://www.wikidata.org/entity/Q50621171'), ('name', 'honorary doctor of the University of Madrid Complutense')]
[('obj', 'http://www.wikidata.org/entity/Q3141777'), ('name', 'Franklin M

In [15]:
# class of Nobel Prize in Physics
queryString = """
SELECT  DISTINCT ?obj ?name 
WHERE {
    wd:Q38104 wdt:P31 ?obj .
    ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q107494520'), ('name', 'physics award')]
1


In [16]:
# look at the properties of Nobel Prize iun Physics
queryString = """
SELECT  DISTINCT ?pr ?name 
WHERE {
    wd:Q38104 ?pr ?obj .
    ?pr sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('pr', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('pr', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('pr', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('pr', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('pr', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('pr', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('pr', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('pr', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('pr', 'http://www.wikidata.org/prop/d

In [17]:
# Nobel Prize iun Physics is part of Nobel Prize
queryString = """
SELECT  DISTINCT ?obj ?name 
WHERE {
    wd:Q38104 wdt:P361 ?obj .
    ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q7191'), ('name', 'Nobel Prize')]
1


In [18]:
### insert the result of TASK 3 in the file
og_uri = "http://www.wikidata.org/entity/Q7191"
og_name = "Nobel Prize"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"3", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 1_0
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_0.json
JSON object updated


## Task 4

In [19]:
# look at Nobel prize in physics winners in Europe
queryString = """
SELECT  DISTINCT ?country ?name COUNT(?obj) AS ?winners 
WHERE {
    ?obj wdt:P166 wd:Q38104 .
    ?obj wdt:P19 ?city.
    ?city wdt:P17 ?country.
    ?country wdt:P30 wd:Q46.
    ?country sc:name ?name.
}
GROUP BY ?country ?name
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q212'), ('name', 'Ukraine'), ('winners', '1')]
[('country', 'http://www.wikidata.org/entity/Q27'), ('name', 'Ireland'), ('winners', '1')]
[('country', 'http://www.wikidata.org/entity/Q28'), ('name', 'Hungary'), ('winners', '2')]
[('country', 'http://www.wikidata.org/entity/Q32'), ('name', 'Luxembourg'), ('winners', '1')]
[('country', 'http://www.wikidata.org/entity/Q756617'), ('name', 'Danish Realm'), ('winners', '2')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('name', 'Sweden'), ('winners', '4')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('name', 'Switzerland'), ('winners', '6')]
[('country', 'http://www.wikidata.org/entity/Q36'), ('name', 'Poland'), ('winners', '7')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('name', 'Netherlands'), ('winners', '9')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium'), ('winners', '1')]
[('country', 'http://www.wikidata.org/entity/Q184'), ('name', '

In [20]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    val = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= val
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"4", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 1_0
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_0.json
JSON object updated


## Task 5

In [21]:
# look at Nobel prize winners in Europe (get the countries)
queryString = """
SELECT DISTINCT ?country ?name
WHERE {
    {
        SELECT COUNT(DISTINCT ?n) AS ?nNobel WHERE{
            ?n wdt:P361 wd:Q7191.
        }
    }
    {
        SELECT  DISTINCT ?country ?name COUNT(DISTINCT ?nobel) AS ?winners 
        WHERE {
            ?nobel wdt:P361 wd:Q7191.
            ?obj wdt:P166 ?nobel .
            ?obj wdt:P19 ?city.
            ?city wdt:P17 ?country.
            ?country sc:name ?name.
        }GROUP BY ?country ?name
    }
FILTER (?winners = ?nNobel).   
} 
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q34'), ('name', 'Sweden')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('name', 'Canada')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('name', 'Austria')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('country', 'http://www.wikidata.org/entity/Q668'), ('name', 'India')]
[('country', 'http://www.wikidata.org/entity/Q20'), ('name', 'Norway')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q36'), ('name', 'Poland')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('name', 'Russia')]
12


In [22]:
obj = [{"uri":r[0][1],"name":r[1][1]} for r in x]
evaluation.add_result(evaluation.get_index_workflow(pt),"4", evaluation.TYPE_SET ,"uri", obj)

The index of this workflow is: 1_0
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_0.json
JSON object updated


## Task 6

In [25]:
# get the number of discoveries for people who won the Nobel Pize in Physics
# filter only whose with at least 3 discoveries
queryString = """
SELECT  ?person ?name COUNT(?disc) AS ?discoveries
WHERE {
    ?disc wdt:P61 ?person .
    ?person wdt:P166 wd:Q38104 .
    ?person sc:name ?name.
}
GROUP BY ?person ?name
HAVING(COUNT(?disc) > 2)
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q193803'), ('name', 'Roger Penrose'), ('discoveries', '6')]
[('person', 'http://www.wikidata.org/entity/Q8753'), ('name', 'Enrico Fermi'), ('discoveries', '6')]
[('person', 'http://www.wikidata.org/entity/Q124013'), ('name', 'Didier Queloz'), ('discoveries', '3')]
[('person', 'http://www.wikidata.org/entity/Q47480'), ('name', 'Paul Dirac'), ('discoveries', '3')]
[('person', 'http://www.wikidata.org/entity/Q937'), ('name', 'Albert Einstein'), ('discoveries', '10')]
[('person', 'http://www.wikidata.org/entity/Q173028'), ('name', 'Emilio G. Segrè'), ('discoveries', '3')]
[('person', 'http://www.wikidata.org/entity/Q123975'), ('name', 'Michel Mayor'), ('discoveries', '12')]
[('person', 'http://www.wikidata.org/entity/Q47285'), ('name', 'J. J. Thomson'), ('discoveries', '3')]
[('person', 'http://www.wikidata.org/entity/Q37463'), ('name', 'Pierre Curie'), ('discoveries', '4')]
[('person', 'http://www.wikidata.org/entity/Q160518'), ('name', 

In [26]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    val = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= val
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"6", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 1_0
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_0.json
JSON object updated


## Task 7

In [27]:
# look at the properties of people who won the Nobel Prize in Physics
queryString = """
SELECT DISTINCT ?pr ?name
WHERE {
    ?person wdt:P166 wd:Q38104 ;
        ?pr ?o.
    ?pr sc:name ?name.
    FILTER(!isLiteral(?o)).
}
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('pr', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('pr', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('pr', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('pr', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('pr', 'http://www.wikidata.org/prop/direct/P1066'), ('name', 'student of')]
[('pr', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('pr', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('pr', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('pr', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('pr', 'http://www

In [28]:
# use employer
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
    ?person wdt:P166 wd:Q38104 ; 
        wdt:P108 ?o.
    ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1047060'), ('name', 'Case Western Reserve University')]
[('o', 'http://www.wikidata.org/entity/Q131252'), ('name', 'University of Chicago')]
[('o', 'http://www.wikidata.org/entity/Q13164'), ('name', 'Moscow State University')]
[('o', 'http://www.wikidata.org/entity/Q13371'), ('name', 'Harvard University')]
[('o', 'http://www.wikidata.org/entity/Q151510'), ('name', 'Heidelberg University')]
[('o', 'http://www.wikidata.org/entity/Q151991'), ('name', 'European Southern Observatory')]
[('o', 'http://www.wikidata.org/entity/Q152171'), ('name', 'University of Bonn')]
[('o', 'http://www.wikidata.org/entity/Q152838'), ('name', 'University of Göttingen')]
[('o', 'http://www.wikidata.org/entity/Q154804'), ('name', 'Leipzig University')]
[('o', 'http://www.wikidata.org/entity/Q165980'), ('name', 'University of Vienna')]
[('o', 'http://www.wikidata.org/entity/Q168751'), ('name', 'Duke University')]
[('o', 'http://www.wikidata.org/entity/Q168756'), ('

In [29]:
# find the class of university
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
    wd:Q15574 wdt:P31 ?o.
    ?o sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q875538'), ('name', 'public university')]
1


In [30]:
# find the class of university
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
    wd:Q875538 wdt:P279 ?o.
    ?o sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3918'), ('name', 'university')]
[('o', 'http://www.wikidata.org/entity/Q23002037'), ('name', 'public educational institution')]
2


In [31]:
# list the subclasses of university
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
    ?o wdt:P279 wd:Q3918.
    ?o sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q15936437'), ('name', 'research university')]
[('o', 'http://www.wikidata.org/entity/Q902104'), ('name', 'private university')]
[('o', 'http://www.wikidata.org/entity/Q1767829'), ('name', 'comprehensive university')]
[('o', 'http://www.wikidata.org/entity/Q265662'), ('name', 'national university')]
[('o', 'http://www.wikidata.org/entity/Q875538'), ('name', 'public university')]
[('o', 'http://www.wikidata.org/entity/Q59537665'), ('name', 'campus university')]
[('o', 'http://www.wikidata.org/entity/Q1371037'), ('name', 'institute of technology')]
[('o', 'http://www.wikidata.org/entity/Q1916585'), ('name', 'medical university')]
[('o', 'http://www.wikidata.org/entity/Q4475845'), ('name', 'university of Marxism-Leninism')]
[('o', 'http://www.wikidata.org/entity/Q5690751'), ('name', 'labor universities')]
[('o', 'http://www.wikidata.org/entity/Q16077796'), ('name', 'vice-ministerial level university')]
[('o', 'http://www.wikidata.org/entity/Q3

In [32]:
# look at the properties of the universities
queryString = """
SELECT DISTINCT ?pr ?name
WHERE {
    ?person wdt:P166 wd:Q38104 ; 
        wdt:P108 ?o.
    ?o wdt:P31/wdt:P279* wd:Q3918.
    ?o ?pr ?x.
    ?pr sc:name ?name.
    FILTER(!isLiteral(?x)).
}
"""

print("Results")
x=run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('pr', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('pr', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('pr', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('pr', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('pr', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location')]
[('pr', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('pr', 'http://www.wikidata.org/prop/direct/P1792'), ('name', 'category of associated people')]
[('pr', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('pr', 'http://www.wikidata.org/prop/direct/P1830'), ('name', 'owner of')]
[('pr', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different fro

In [33]:
# find the physicists who won Nobel Prizes and had been employed for the university in the city where they were born
queryString = """
SELECT DISTINCT ?person  ?pName  ?o ?name 
WHERE {
    ?person wdt:P166 wd:Q38104 ;
        wdt:P19 ?city;
        wdt:P108 ?o.
    ?o wdt:P31/wdt:P279* wd:Q3918;
        wdt:P131 ?city.
    ?o sc:name ?name.
    ?person sc:name ?pName.
}
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q193496'), ('pName', 'James Cronin'), ('o', 'http://www.wikidata.org/entity/Q131252'), ('name', 'University of Chicago')]
[('person', 'http://www.wikidata.org/entity/Q71013'), ('pName', 'Theodor W. Hänsch'), ('o', 'http://www.wikidata.org/entity/Q151510'), ('name', 'Heidelberg University')]
[('person', 'http://www.wikidata.org/entity/Q9130'), ('pName', 'Erwin Schrödinger'), ('o', 'http://www.wikidata.org/entity/Q165980'), ('name', 'University of Vienna')]
[('person', 'http://www.wikidata.org/entity/Q123022'), ('pName', 'Felix Bloch'), ('o', 'http://www.wikidata.org/entity/Q11942'), ('name', 'ETH Zürich')]
[('person', 'http://www.wikidata.org/entity/Q44286'), ('pName', 'Rudolf Mössbauer'), ('o', 'http://www.wikidata.org/entity/Q157808'), ('name', 'Technical University of Munich')]
[('person', 'http://www.wikidata.org/entity/Q188128'), ('pName', 'Alexei Abrikosov'), ('o', 'http://www.wikidata.org/entity/Q1472245'), ('name', 'Bauman Mosc